In [10]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import os
from os.path import join as oj
from sklearn import metrics
from sklearn.utils.multiclass import unique_labels
from sklearn.metrics import confusion_matrix
import numpy as np
from collections import Counter
from copy import deepcopy
from sklearn import metrics
plt.style.use('dark_background')
import pandas as pd
from colorama import Fore
import pickle as pkl
from tqdm import tqdm
import data, viz
from style import *
out_dir = 'results/jul22_19' # jun17_1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# compare all models

In [11]:
def load_results(out_dir):
    r = []
    for fname in tqdm(os.listdir(out_dir)):
        d = pkl.load(open(oj(out_dir, fname), 'rb'))
        d['param_name'] = fname[:-4]
        r.append(pd.Series(d))
    r = pd.concat(r, axis=1, sort=False).T.infer_objects()
    r = r.reindex(sorted(r.columns, reverse=True), axis=1) # sort the column names
    r = r.round(3)
    r = r.set_index('param_name')
    return r

results = load_results(out_dir)
# print(results.keys())

100%|██████████| 62/62 [00:00<00:00, 510.44it/s]


In [12]:
results.head()

,tp_test2_folds,tp_test2,tp_test1_folds,tp_test1,tp_cv_folds,tp_cv,tn_test2_folds,tn_test2,tn_test1_folds,tn_test1,...,balanced_accuracy_test1,balanced_accuracy_cv_folds,balanced_accuracy_cv,balacing_ratio,accuracy_test2_folds,accuracy_test2,accuracy_test1_folds,accuracy_test1,accuracy_cv_folds,accuracy_cv
param_name,,,,,,,,,,,,,,,,,,,,,
dt_select_stab_lasso=22_sample_weights=750,"[11, 16, 14, 14]",55,"[0, 8, 5, 6]",19,"[0, 6, 4, 7]",17,"[2103, 2080, 2089, 2093]",8365,"[3895, 3898, 3909, 3893]",15595,...,0.528,"[0.4955730447614363, 0.564237024565342, 0.5578...",0.560,750,"[0.9661791590493601, 0.9579524680073126, 0.961...",0.962,"[0.9710795312889554, 0.9738219895287958, 0.975...",0.973,"[0.9791059280855199, 0.9709645669291339, 0.976...",0.976
logistic_select_lasso=22_sample_weights=1000,"[60, 58, 60, 59]",237,"[64, 64, 64, 58]",250,"[25, 40, 31, 26]",122,"[734, 971, 747, 1113]",3565,"[1560, 2094, 1488, 2330]",7472,...,0.665,"[0.7031480570585342, 0.7263028049621757, 0.685...",0.710,1000,"[0.36288848263254114, 0.4702925045703839, 0.36...",0.434,"[0.4048865619546248, 0.5380204437796061, 0.386...",0.481,"[0.413508260447036, 0.531003937007874, 0.41113...",0.489
logistic_select_rf=10_sample_weights=1000,"[62, 62, 62, 62]",248,"[71, 73, 73, 69]",286,"[25, 41, 32, 26]",124,"[157, 229, 48, 496]",930,"[401, 681, 110, 997]",2189,...,0.559,"[0.5494343334972946, 0.5594490628690355, 0.515...",0.547,1000,"[0.10009140767824497, 0.1329981718464351, 0.05...",0.135,"[0.11767638992769883, 0.18798304662179008, 0.0...",0.154,"[0.1098153547133139, 0.18208661417322836, 0.04...",0.154
logistic_select_lasso=5_sample_weights=750,"[62, 62, 62, 62]",248,"[72, 72, 72, 72]",288,"[25, 43, 32, 30]",130,"[0, 0, 0, 0]",0,"[17, 17, 17, 17]",68,...,0.495,"[0.5019675356615839, 0.5002513826043238, 0.501...",0.502,750,"[0.0283363802559415, 0.0283363802559415, 0.028...",0.028,"[0.02218898030416355, 0.02218898030416355, 0.0...",0.022,"[0.016034985422740525, 0.021653543307086614, 0...",0.019
logistic_select_lasso=7_sample_weights=1000,"[61, 61, 61, 61]",244,"[70, 70, 70, 70]",280,"[25, 43, 32, 30]",130,"[184, 184, 184, 184]",736,"[128, 128, 128, 128]",512,...,0.496,"[0.5169699950811608, 0.5153343388637506, 0.517...",0.517,1000,"[0.11197440585009141, 0.11197440585009141, 0.1...",0.112,"[0.0493642483171279, 0.0493642483171279, 0.049...",0.049,"[0.0456754130223518, 0.051181102362204724, 0.0...",0.051


## look at prediction metrics

Most important thing is to have very low false negatives (very close to zero). After that, we also want to minimize the false positives.

In [17]:
# prev rule: sensitivity 0.97, specificity 0.425
r = results
# print(r.keys())
# r = r[[k for k in r if not 'std' in k]]
# r = r[[k for k in r if not '_test' in k]]
# r = r[[k for k in r if not '_f' in k]]
# r = r[r.index.str.contains('ros')] # only use random sampling
# r = r.sort_values(by=['balanced_accuracy'], ascending=False)

r = r[r['sensitivity_cv'] > 0.8] # what fraction of true pos do we return?
r = r[r['specificity_cv'] > 0.4] # what fraction of true negs do we return?

r = r.sort_values(by=['sensitivity_cv', 'specificity_cv'], ascending=False)
# r = r[['fn', 'tn', 'fp', 'tp', 'specificity', 'sensitivity', 'roc_auc', 'precision',
#        'f1', 'balanced_accuracy', 'accuracy']]
r = r[['sensitivity_cv', 'specificity_cv', # 'precision',
       'sensitivity_test1', 'specificity_test1',
       'sensitivity_test2', 'specificity_test2',
       'fn_cv', 'tn_cv', 'fp_cv', 'tp_cv',
       'precision_cv', 'roc_auc_cv', 
       'f1_cv', 'balanced_accuracy_cv', 'accuracy_cv',
       'fn_test1', 'tn_test1', 'fp_test1', 'tp_test1',
       'fn_test2', 'tn_test2', 'fp_test2', 'tp_test2']]
# r.style.background_gradient(cmap='viridis', axis=None) # all values on same cmap
# r = r[r.index == 'logistic_select_lasso=27_ros=100']
# r.style.background_gradient(cmap='viridis', axis=0) # columns differently colored
# r.style.apply(viz.highlight_max, subset=[k for k in r if not 'std' in k])

In [18]:
r

,sensitivity_cv,specificity_cv,sensitivity_test1,specificity_test1,sensitivity_test2,specificity_test2,fn_cv,tn_cv,fp_cv,tp_cv,...,balanced_accuracy_cv,accuracy_cv,fn_test1,tn_test1,fp_test1,tp_test1,fn_test2,tn_test2,fp_test2,tp_test2
param_name,,,,,,,,,,,,,,,,,,,,,
logistic_select_stab_lasso=6_sample_weights=500,0.969,0.522,0.873,0.514,0.984,0.509,4,4129,3774,126,...,0.746,0.530,37,8101,7651,255,4,4325,4179,244
logistic_select_stab_lasso=7_sample_weights=750,0.969,0.509,0.880,0.504,0.972,0.503,4,4026,3877,126,...,0.739,0.517,35,7943,7809,257,7,4278,4226,241
logistic_select_stab_lasso=6_sample_weights=750,0.969,0.504,0.887,0.500,0.984,0.493,4,3983,3920,126,...,0.737,0.512,33,7869,7883,259,4,4193,4311,244
logistic_select_stab_lasso=6_sample_weights=1000,0.969,0.504,0.887,0.500,0.984,0.493,4,3983,3920,126,...,0.737,0.512,33,7869,7883,259,4,4193,4311,244
logistic_select_stab_lasso=5_sample_weights=500,0.954,0.531,0.863,0.524,0.984,0.516,6,4193,3710,124,...,0.742,0.537,40,8251,7501,252,4,4388,4116,244
logistic_select_stab_lasso=10_sample_weights=1000,0.954,0.434,0.897,0.424,0.976,0.386,6,3428,4475,124,...,0.694,0.442,30,6682,9070,262,6,3281,5223,242
logistic_select_stab_lasso=10_sample_weights=750,0.946,0.492,0.887,0.483,0.972,0.445,7,3886,4017,123,...,0.719,0.499,33,7605,8147,259,7,3788,4716,241
logistic_select_lasso=22_sample_weights=1000,0.938,0.482,0.856,0.474,0.956,0.419,8,3807,4096,122,...,0.710,0.489,42,7472,8280,250,11,3565,4939,237
logistic_select_stab_lasso=22_sample_weights=1000,0.938,0.482,0.856,0.474,0.956,0.419,8,3807,4096,122,...,0.710,0.489,42,7472,8280,250,11,3565,4939,237


**look at feat importances**

In [23]:
r = results
best_model = 'logistic_select_rf=10_sample_weights=1000' #logistic_select_rf=5_smote=5'
# print(r.keys())
# r.style.apply(viz.highlight_max, subset=[k for k in r if not 'std' in k])
r = r.sort_values('balanced_accuracy', ascending=False)
keys = [k for k in r if '_f' in k]
keys_remapped = {k: k.replace('_f', '') for k in keys}
r = r[keys].rename(columns=keys_remapped)
# r = r.sort_values('lifetime')
# r = r[r.index.str.contains('35')]
# r = r[r.index.str.contains('11')]
# r = r.rename(columns={'mean_square_displacement': 'msd', 'total_displacement': 'td'})

# r = r[r.index.str.contains('20')]
r = r[r.index.str.contains(best_model)]

r = r[[k for k in r if not 'std' in k]]


def rank(r):
    '''Rank feature importances appropriately
    '''
    r = r.abs()
    r = r.rank(axis=1, ascending=False, method='min')
    return r
# 
# r = rank(r)
r = r.reindex(r.mean().sort_values(ascending=True).index, axis=1) # sort cols by mean rank
idxs = r.index
# r.insert(0, 'balanced_acc', results.loc[idxs]['balanced_accuracy'])
# r = r.sort_values('balanced_acc', ascending=False)
# subset = list(r.keys())
# subset.remove('balanced_acc')
# r.to_html('table.html')
# r = r.fillna(0).style.background_gradient(cmap='viridis_r', axis=1) #, subset=subset) # rows differently colored


r = r.transpose()
r = r.reindex(r[best_model].abs().sort_values(ascending=False).index) # sort by abs value
# r = r.sort_values(by='logistic_20_ros=1')
r = r.fillna(0).style.background_gradient(cmap=cm) #, axis=0) #, subset=subset) # rows differently colored
# r = background_gradient(r)
r

model_type,logistic_select_rf=10_sample_weights=1000
RtCostalTender,1.079
AbdTrauma_or_SeatBeltSign_yes,0.825
AbdTenderDegree_Severe,0.631
InitHeartRate,0.391
AbdDistention_yes,0.389
InitSysBPRange,-0.323
VomitWretch_yes,0.272
Age,0.197
MOI_Motor vehicle collision,0.197
ThoracicTrauma_yes,-0.017


# misc analysis

In [ ]:
def count_common_feats(m_cv, model_type='tree'):
    if model_type == 'tree':
        feats_used = []
        for i in range(len(m_cv['estimator'])):
            m_fit = m_cv['estimator'][i]
            feats_used += list(ks[m_fit.feature_importances_ != 0])
    elif model_type == 'logistic':
        feats_used = []
        for i in range(len(m_cv['estimator'])):
            m_fit = m_cv['estimator'][i]
            num_feats = 5
            # get top num_feats features with biggest bsolute weights
            idxs = np.abs(m_fit.coef_).flatten().argsort()[-num_feats:][::-1]
            feats_used += list(ks[idxs])
    return sorted(dict(Counter(feats_used)).items(), key=lambda kv: kv[1], reverse=True)
        
count_common_feats(m_cv, model_type)

In [ ]:
# plot a tree
m_fit = m_cv['estimator'][0]
plt.figure(dpi=300)
plot_tree(m_fit, feature_names=ks)
plt.show()